# Problema

Hacer predicciones del coste del seguro

## Instrucciones

 Utilizar el dataset (insurance.csv) para entrenar un modelo de regresión capaz de predecir el valor del seguro en función de las características del cliente. Realizar limpieza, preprocesado modelado y testeo del modelo aportando conclusiones de todos estos pasos.

# El set de datos

* age: age of primary beneficiary

* sex: insurance contractor gender, female, male

* bmi: Body mass index, providing an understanding of body, weights that are relatively high or low relative to height,
objective index of body weight (kg / m ^ 2) using the ratio of height to weight, ideally 18.5 to 24.9

* children: Number of children covered by health insurance / Number of dependents

* smoker: Smoking

* region: the beneficiary's residential area in the US, northeast, southeast, southwest, northwest.

* charges: Individual medical costs billed by health insurance



In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler


In [3]:
ruta = "insurance.csv"
data = pd.read_csv(ruta)

In [4]:
print(data.shape)
data.head()

(1338, 7)


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


# Objetivo

Generar un model de regresión capaz de predecir el valor del seguro en base a las características del cliente.

* Aplicar las técnicas oportunas de procesamiento de datos (lipieza, nans, escalado...)

* Valorar diferentes modelos de regresión (linear regressor, polynomial, ridge, lasso, elastic, decission tree y random forest)

* Comparación entre modelos (dividir el dataset en train y test, entrenar con el train y evaluar con el test)

* Métricas (todas, y que aporta cada una)

* Conclusiones finales

## Implementación

#### Limpieza de datos

In [5]:
data.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
charges     float64
dtype: object

In [6]:
data.describe(include='all')

,age,sex,bmi,children,smoker,region,charges
count,1338.000000,1338,1338.000000,1338.000000,1338,1338,1338.000000
unique,NaN,2,NaN,NaN,2,4,NaN
top,NaN,male,NaN,NaN,no,southeast,NaN
freq,NaN,676,NaN,NaN,1064,364,NaN
mean,39.207025,NaN,30.663397,1.094918,NaN,NaN,13270.422265
std,14.049960,NaN,6.098187,1.205493,NaN,NaN,12110.011237
min,18.000000,NaN,15.960000,0.000000,NaN,NaN,1121.873900
25%,27.000000,NaN,26.296250,0.000000,NaN,NaN,4740.287150
50%,39.000000,NaN,30.400000,1.000000,NaN,NaN,9382.033000
75%,51.000000,NaN,34.693750,2.000000,NaN,NaN,16639.912515


In [7]:
nan_por_columna = data.isna().sum()
print(nan_por_columna)

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


In [8]:
# Codificación de variables categóricas
categorical_cols = data.select_dtypes(include=['object']).columns
print(categorical_cols)

Index(['sex', 'smoker', 'region'], dtype='object')


In [9]:
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True).astype(int)
print(data_encoded.head)

<bound method NDFrame.head of       age  bmi  children  charges  sex_male  smoker_yes  region_northwest  \
0      19   27         0    16884         0           1                 0   
1      18   33         1     1725         1           0                 0   
2      28   33         3     4449         1           0                 0   
3      33   22         0    21984         1           0                 1   
4      32   28         0     3866         1           0                 1   
...   ...  ...       ...      ...       ...         ...               ...   
1333   50   30         3    10600         1           0                 1   
1334   18   31         0     2205         0           0                 0   
1335   18   36         0     1629         0           0                 0   
1336   21   25         0     2007         0           0                 0   
1337   61   29         0    29141         0           1                 1   

      region_southeast  region_southwest  
0 

In [10]:
# Escalado de variables numéricas
numeric_cols = data_encoded.select_dtypes(include=['int64', 'float64']).columns
scaler = MinMaxScaler()  # Normalización estándar (media = 0, desviación estándar = 1)
data_encoded[numeric_cols] = scaler.fit_transform(data_encoded[numeric_cols])
print(data_encoded.head())

        age       bmi  children   charges  sex_male  smoker_yes  \
0  0.021739  0.315789       0.0  0.251608       0.0         1.0   
1  0.000000  0.473684       0.2  0.009641       1.0         0.0   
2  0.217391  0.473684       0.6  0.053121       1.0         0.0   
3  0.326087  0.184211       0.0  0.333014       1.0         0.0   
4  0.304348  0.342105       0.0  0.043816       1.0         0.0   

   region_northwest  region_southeast  region_southwest  
0               0.0               0.0               1.0  
1               0.0               1.0               0.0  
2               0.0               1.0               0.0  
3               1.0               0.0               0.0  
4               1.0               0.0               0.0  


In [11]:
from sklearn.model_selection import train_test_split

# Separar las características (X) y la variable objetivo (y)
X = data_encoded.drop('charges', axis=1) 
y = data_encoded['charges']

# Dividir el dataset en train y test (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Lista para almacenar los resultados de los modelos
results = []

# Función para entrenar y evaluar modelos en train y test
def train_and_evaluate(model, model_name, X_train, X_test, y_train, y_test):
    # Entrenar el modelo
    model.fit(X_train, y_train)
    
    # Predecir en train y test
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Calcular métricas en train
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mse_train = mean_squared_error(y_train, y_pred_train)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(y_train, y_pred_train)
    
    # Calcular métricas en test
    mae_test = mean_absolute_error(y_test, y_pred_test)
    mse_test = mean_squared_error(y_test, y_pred_test)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(y_test, y_pred_test)
    
    # Almacenar resultados
    results.append({
        'Model': model_name,
        'MAE Train': mae_train,
        'MAE Test': mae_test,
        'MSE Train': mse_train,
        'MSE Test': mse_test,
        'RMSE Train': rmse_train,
        'RMSE Test': rmse_test,
        'R2 Train': r2_train,
        'R2 Test': r2_test
    })
    
    # Imprimir resultados
    print(f"Modelo: {model_name}")
    print("Métricas en Train:")
    print(f"MAE: {mae_train}, MSE: {mse_train}, RMSE: {rmse_train}, R2: {r2_train}")
    print("Métricas en Test:")
    print(f"MAE: {mae_test}, MSE: {mse_test}, RMSE: {rmse_test}, R2: {r2_test}")
    print("-" * 50)

# Modelos a evaluar
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42)
}

# Entrenar y evaluar cada modelo
for name, model in models.items():
    train_and_evaluate(model, name, X_train, X_test, y_train, y_test)

# Modelo de regresión polinómica
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

linear_model = LinearRegression()
train_and_evaluate(linear_model, 'Polynomial Regression', X_train_poly, X_test_poly, y_train, y_test)


Modelo: Linear Regression
Métricas en Train:
MAE: 0.06714050214591512, MSE: 0.009501118761605525, RMSE: 0.09747368240507549, R2: 0.7416341270978415
Métricas en Test:
MAE: 0.06666137361010166, MSE: 0.008552183156987718, RMSE: 0.09247801445201836, R2: 0.7837888448800692
--------------------------------------------------
Modelo: Ridge Regression
Métricas en Train:
MAE: 0.06702559467086737, MSE: 0.009503596006965955, RMSE: 0.09748638882924095, R2: 0.7415667628562189
Métricas en Test:
MAE: 0.06655803431084878, MSE: 0.008569676567426692, RMSE: 0.09257254759066909, R2: 0.783346586990058
--------------------------------------------------
Modelo: Lasso Regression
Métricas en Train:
MAE: 0.1436663420503868, MSE: 0.036773892212937635, RMSE: 0.19176520073500727, R2: 0.0
Métricas en Test:
MAE: 0.1531282311795573, MSE: 0.03959113352892433, RMSE: 0.19897520832738014, R2: -0.0009192454913706793
--------------------------------------------------
Modelo: ElasticNet Regression
Métricas en Train:
MAE: 0.1

In [13]:
# Convertir los resultados a un DataFrame para una mejor visualización
results_df = pd.DataFrame(results)
print(results_df)

                   Model  MAE Train  MAE Test  MSE Train  MSE Test  \
0      Linear Regression   0.067141  0.066661   0.009501  0.008552   
1       Ridge Regression   0.067026  0.066558   0.009504  0.008570   
2       Lasso Regression   0.143666  0.153128   0.036774  0.039591   
3  ElasticNet Regression   0.143666  0.153128   0.036774  0.039591   
4          Decision Tree   0.000983  0.053386   0.000117  0.011972   
5          Random Forest   0.017193  0.043556   0.000977  0.005659   
6  Polynomial Regression   0.046041  0.043447   0.005798  0.005313   

   RMSE Train  RMSE Test  R2 Train   R2 Test  
0    0.097474   0.092478  0.741634  0.783789  
1    0.097486   0.092573  0.741567  0.783347  
2    0.191765   0.198975  0.000000 -0.000919  
3    0.191765   0.198975  0.000000 -0.000919  
4    0.010821   0.109418  0.996816  0.697326  
5    0.031259   0.075224  0.973429  0.856940  
6    0.076147   0.072887  0.842322  0.865691  


## ¿Qué aporta cada metrica?

MAE (Mean Absolute Error - Error Absoluto Medio)
   

¿Qué aporta?


-Mide el error promedio absoluto entre los valores reales y los predichos.
-Es fácil de interpretar, ya que está en las mismas unidades que la variable objetivo.
-No penaliza tanto los errores grandes como el MSE o RMSE.
-Es robusto frente a valores atípicos (outliers).


MSE (Mean Squared Error - Error Cuadrático Medio)
   
   
¿Qué aporta?


Mide el error cuadrático promedio entre los valores reales y los predichos.
Penaliza más los errores grandes (debido al cuadrado), lo que es útil si quieres evitar predicciones muy alejadas de los valores reales.
Es sensible a valores atípicos (outliers), ya que los errores grandes se amplifican al elevarlos al cuadrado.


RMSE (Root Mean Squared Error - Raíz del Error Cuadrático Medio)
   

¿Qué aporta?

-Es la raíz cuadrada del MSE, por lo que está en las mismas unidades que la variable objetivo.
-Combina las ventajas del MSE (penaliza errores grandes) con una interpretación más intuitiva (mismas unidades que y).
-También es sensible a valores atípicos.


R² (Coeficiente de Determinación)

¿Qué aporta?

-Mide la proporción de la varianza en la variable dependiente (y) que es explicada por el modelo.
-Su valor oscila entre 0 y 1:
-Es útil para comparar modelos: un R² más alto indica un mejor ajuste.

# Conclusiones

**Conclusión basada en los resultados**

**Mejor rendimiento en test:**

Random Forest y Regresión Polinómica son los modelos con mejor rendimiento:

R² alto (0.857 y 0.866 respectivamente).

Errores bajos (MAE: ~0.043-0.044, RMSE: ~0.075-0.073).

**Modelos con sobreajuste o bajo rendimiento:**

Lasso y ElasticNet tienen un R² negativo en test (-0.0009), lo que indica que son peores que un modelo que simplemente prediga la media. Esto sugiere que la regularización aplicada es demasiado agresiva o que no capturan relaciones útiles en los datos.

Árbol de Decisión tiene un R² menor (0.697) en comparación con Random Forest y Regresión Polinómica, lo que refleja su tendencia a sobreajustar sin técnicas de ensemble.

**Regresión Polinómica destaca:**

El R² más alto (0.866) sugiere que las relaciones no lineales (capturadas por términos polinómicos) son clave en este problema. Sin embargo, es importante validar si hay sobreajuste comparando métricas en train y test.
